# Toxic: SHAP and XGBoost 

In [1]:

import shap
import xgboost
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
# import cupy as cp


## Load dataset


In [2]:
combinedbuoyspath = "../data/data/ToxicsChemicalContaminantHUC8new.csv"
# New file with deleted row no 131050, for 'new' file row no 275590

# data = pd.read_csv(combinedbuoyspath, on_bad_lines='skip')
data = pd.read_csv(combinedbuoyspath,sep=',')
data.info()

C:\Users\pushk\AppData\Local\Temp\ipykernel_42620\2154918495.py:5: DtypeWarning: Columns (0,18,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(combinedbuoyspath,sep=',')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321572 entries, 0 to 321571
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   HUC8               321571 non-null  object 
 1   Station            321570 non-null  object 
 2   Latitude           321570 non-null  float64
 3   Longitude          321570 non-null  float64
 4   LLDatum            321570 non-null  object 
 5   MediaType          321570 non-null  object 
 6   SampleDateTime     321570 non-null  object 
 7   RefDocId           321570 non-null  object 
 8   ChemicalType       321570 non-null  object 
 9   ChemicalName       321570 non-null  object 
 10  ReportedValue      256799 non-null  float64
 11  Unit               321570 non-null  object 
 12  Qualifier          149442 non-null  object 
 13  ValueType          321570 non-null  object 
 14  DLLower            258605 non-null  float64
 15  MethodId           321570 non-null  object 
 16  So

### Inspect column names

In [3]:
data.columns

Index(['HUC8', 'Station', 'Latitude', 'Longitude', 'LLDatum', 'MediaType',
       'SampleDateTime', 'RefDocId', 'ChemicalType', 'ChemicalName',
       'ReportedValue', 'Unit', 'Qualifier', 'ValueType', 'DLLower',
       'MethodId', 'Source', 'MethodDescription', 'SampleDepth', 'LatinName',
       'CommonName', 'Details', 'Unnamed: 22'],
      dtype='object')

## Chemical names need to be their own columns

In [4]:
data_r = data.pivot_table(
    index=["SampleDateTime"], columns="ChemicalName", values="ReportedValue", aggfunc="first"
).reset_index()

In [5]:
pivotcols = data_r.columns
print(pivotcols.shape)
data_r

(744,)


ChemicalName,SampleDateTime,% LIPID,% MOISTURE,"1,1,1-TRICHLOROETHANE","1,1,2,2-TETRACHLOROETHANE","1,1,2-TRICHLOROETHANE","1,1-DICHLOROETHANE","1,2,3,4-TETRACHLOROBENZENE","1,2,4,5-TETRACHLOROBENZENE","1,2,4-TRICHLOROBENZENE",...,URANIUM,Unknown tetra-BDE,"Unknown tetra-BDE + PCB-171,156",VANADIUM,VINYL CHLORIDE,VINYLIDENE CHLORIDE,"VINYLIDENE CHLORIDE, TOTAL",WEIGHT,XYLENE (MIXED ISOMERS),ZINC
0,1/1/1993 13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0
1,1/10/1995 10:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1/10/1995 10:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1/10/1995 10:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/10/1995 10:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12394,9/9/1998 15:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12395,9/9/1998 23:59,NaN,NaN,NaN,NaN,NaN,NaN,0.145677,0.347083,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,301.0
12396,9/9/1999 13:50,NaN,56.1,NaN,NaN,NaN,NaN,0.130958,0.060863,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.0
12397,9/9/1999 15:06,NaN,64.3,NaN,NaN,NaN,NaN,0.025546,0.277368,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.5


In [6]:
def combine_date_time_strings(df, originaldtcolumn):
    # Combine date and time strings
    combined_col = df[originaldtcolumn]
    # Convert the combined string to datetime
    datetime_col = "Sample"

    df[datetime_col] = pd.to_datetime(combined_col, errors="coerce")
    df[datetime_col + "_year"] = df[datetime_col].dt.year
    df[datetime_col + "_month"] = df[datetime_col].dt.month
    df[datetime_col + "_day"] = df[datetime_col].dt.day
    df[datetime_col + "_hour"] = df[datetime_col].dt.hour
    df[datetime_col + "_minute"] = df[datetime_col].dt.minute
    df[datetime_col + "_second"] = df[datetime_col].dt.second
    newcolnames = [
        f"{datetime_col}_year",
        f"{datetime_col}_month",
        f"{datetime_col}_day",
        f"{datetime_col}_hour",
        f"{datetime_col}_minute",
        f"{datetime_col}_second",
    ]
    # Drop the original date, time, and combined datetime columns
    df.drop(columns=[originaldtcolumn, datetime_col], inplace=True)

    return df, newcolnames

In [7]:
columns_to_exclude = ["ChemicalName", "ReportedValue", "Unit", "Qualifier"]
unique_columns = [col for col in data.columns if col not in columns_to_exclude]
df_unique = data[unique_columns].drop_duplicates(subset="SampleDateTime")
exclude_from_pivoted = ["ChemicalName", "ReportedValue", "Unit", "Qualifier"]
data_cleana, newcols = combine_date_time_strings(data_r, "SampleDateTime")
newcols


['Sample_year',
 'Sample_month',
 'Sample_day',
 'Sample_hour',
 'Sample_minute',
 'Sample_second']

In [8]:
data_clean = data.replace([np.inf, -np.inf], np.nan)
missing_percentage = data_clean.isnull().mean()
print(missing_percentage)
clean_columns = data_clean.columns.tolist()

# Drop columns where more than 95% of the data is missing
columns_to_drop = missing_percentage[missing_percentage > 0.90].index
print(columns_to_drop)
# data_clean = data_clean.dropna(subset=['Chlorophyll'])

HUC8                 0.000003
Station              0.000006
Latitude             0.000006
Longitude            0.000006
LLDatum              0.000006
MediaType            0.000006
SampleDateTime       0.000006
RefDocId             0.000006
ChemicalType         0.000006
ChemicalName         0.000006
ReportedValue        0.201426
Unit                 0.000006
Qualifier            0.535277
ValueType            0.000006
DLLower              0.195810
MethodId             0.000006
Source               0.000006
MethodDescription    0.000006
SampleDepth          0.710917
LatinName            0.000081
CommonName           0.640196
Details              0.999779
Unnamed: 22          0.999779
dtype: float64
Index(['Details', 'Unnamed: 22'], dtype='object')


In [9]:
data_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321572 entries, 0 to 321571
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   HUC8               321571 non-null  object 
 1   Station            321570 non-null  object 
 2   Latitude           321570 non-null  float64
 3   Longitude          321570 non-null  float64
 4   LLDatum            321570 non-null  object 
 5   MediaType          321570 non-null  object 
 6   SampleDateTime     321570 non-null  object 
 7   RefDocId           321570 non-null  object 
 8   ChemicalType       321570 non-null  object 
 9   ChemicalName       321570 non-null  object 
 10  ReportedValue      256799 non-null  float64
 11  Unit               321570 non-null  object 
 12  Qualifier          149442 non-null  object 
 13  ValueType          321570 non-null  object 
 14  DLLower            258605 non-null  float64
 15  MethodId           321570 non-null  object 
 16  So